In [1]:
from blinker import signal

def send_sig(signal_name, **kwargs):
    s = signal("events")
    return s.send(signal_name,**kwargs)


events = signal("events")
# @events.connect_via("test")
# def on_test(sender, message: str):
#     print(f"Received notification: {message}")

@events.connect
def log(sender, **kwargs):
    yield f"Logging signal: Sender: {sender} Args: {kwargs}"
    print(f"Logging signal: Sender: {sender} Args: {kwargs}")

In [2]:
results = send_sig("test2", message="Hello")
# send_sig("test3", **{"a": 2, "b": 2})
# signal("test2").send("test2", message="Hello THERE!!")
results

[(<function __main__.log(sender, **kwargs)>,
  <generator object log at 0x7f49d12e1e00>)]

In [3]:
class Notifcation:
    def __init__(self, message: str):
        events.send("test", message=message)


tester = Notifcation("Hello from Tester")

In [4]:
Notifcation("Hello from Second Tester")


In [5]:
from blinker import ANY, signal
from typing import TypeVar, Any, Callable
import collections.abc as c
test2 = signal("test2")

F = TypeVar("F", bound=c.Callable[..., Any])

def event(signal_name: str, sender: Any = ANY, weak: bool = True) -> c.Callable[[F], F]:
    sig = signal(signal_name)
    def decorator(fn):
        sig.connect(fn, sender, weak)
        return fn
    return decorator


In [6]:
@event("test2")
def on_test(sender, message: str):
    print(f"Received notification2: {message}")

In [7]:
from blinker.base import make_id
test2.send("test2", message="Hello THERE!!")
for test in test2.receivers:
    print(test)
make_id(on_test)
test2.receivers[make_id(on_test)]

Received notification2: Hello THERE!!
139955018982688


<weakref at 0x7f49d12c3f10; to 'function' at 0x7f49d130e520 (on_test)>

In [8]:
from collections import deque

queue = deque([])  # list of tuples is fine

queue.extend([("job3", 14), ("job4", 15)])

print(queue)

while queue:
    item = queue.popleft()  # fast O(1)
    print("Processing:", item)
    
print(queue)

deque([('job3', 14), ('job4', 15)])
Processing: ('job3', 14)
Processing: ('job4', 15)
deque([])


In [9]:
signal("events").send("user", message="Hello")

[(<function __main__.log(sender, **kwargs)>,
  <generator object log at 0x7f49d12e1fc0>)]

In [10]:
from uuid import uuid4
str(uuid4().hex)

'e1c24731b1e7411c8208503f27f52c3a'

In [11]:
class AttrDict(dict):
    "`dict` subclass that also provides access to keys as attrs"
    def __getattr__(self,k): return self[k] if k in self else None
    def __setattr__(self, k, v): (self.__setitem__,super().__setattr__)[k[0]=='_'](k,v)
    def __dir__(self): return super().__dir__() + list(self.keys())
    def copy(self): return AttrDict(**self)


signals = AttrDict(message="Hello")
signals.message
signals.name = "John"
signals.name or "No name"



'John'

In [12]:
from rusty_tags.events import event
import asyncio

evt = event("TestEvent")

@evt.connect
def sync(sender, **kwargs):
    print(f"Logging sync: Sender: {sender} Args: {kwargs}")
    return "Hello sync"

@evt.connect
def sync_gen(sender, **kwargs):
    print(f"Logging sync_gen: Sender: {sender} Args: {kwargs}")
    yield "Hello "
    yield "sync"
    yield "generator"


@evt.connect
async def async_log(sender, **kwargs):
    print(f"Logging async_log: Sender: {sender} Args: {kwargs}")
    return "Hello async"

@evt.connect
async def async_gen(sender, **kwargs):
    print(f"Logging async_gen: Sender: {sender} Args: {kwargs}")
    yield "Hello "
    yield "async"
    yield "generator"

async_results = await evt.emit_async("test", message="Hello")
sync_results = evt.emit("test", message="Hello")
print("emit_async results: \n",async_results)
print("emit results: \n",sync_results)

Logging sync: Sender: test Args: {'message': 'Hello'}
Logging async_log: Sender: test Args: {'message': 'Hello'}
Logging async_gen: Sender: test Args: {'message': 'Hello'}
Logging sync_gen: Sender: test Args: {'message': 'Hello'}
Logging sync: Sender: test Args: {'message': 'Hello'}
Logging sync_gen: Sender: test Args: {'message': 'Hello'}
emit_async results: 
 [['Hello sync'], ['Hello async'], ['Hello ', 'async', 'generator'], ['Hello ', 'sync', 'generator']]
emit results: 
 [['Hello sync'], <Task pending name='Task-10' coro=<Event._schedule_async.<locals>.consume() running at /home/ndendic/WebDev/rustyTags/rusty_tags/events.py:80>>, <Task pending name='Task-11' coro=<Event._schedule_async.<locals>.consume() running at /home/ndendic/WebDev/rustyTags/rusty_tags/events.py:80>>, ['Hello ', 'sync', 'generator']]
Logging async_log: Sender: test Args: {'message': 'Hello'}
Logging async_gen: Sender: test Args: {'message': 'Hello'}
